# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import json
import numpy as np
import requests
import gmaps
import os
# Import API key
from api_keys import (weather_api_key, g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# import the weather data for the cities from WeatherPy homework
# Load the csv exported in Part I to a DataFrame
weather_cities_data = pd.read_csv("../WeatherPy/weather_data.csv")
weather_cities_data


,City,Country,Date,Latitude,Longitude,Temperature,Max Temperature,Humidity,Clouds,Wind Speed
0,Mitú,CO,1.627416e+09,1.1983,-70.1733,79.00,79.00,92.0,100.0,0.76
1,East London,ZA,1.627416e+09,-33.0153,27.9116,73.15,73.15,30.0,0.0,8.08
2,Hobart,AU,1.627416e+09,-42.8794,147.3294,54.70,55.92,68.0,61.0,4.00
3,Santa Isabel Rio Negro,BR,1.627416e+09,-0.4139,-65.0192,79.84,79.84,80.0,100.0,2.48
4,Bellavista,PE,1.627416e+09,-7.0561,-76.5911,90.88,92.48,55.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...
713,Derzhavīnsk,KZ,1.627416e+09,51.1000,66.3167,60.84,60.84,46.0,5.0,5.55
714,Boshnyakovo,RU,1.627416e+09,49.6333,142.1667,65.64,65.64,90.0,97.0,2.55
715,Las Tunas,CU,1.627416e+09,20.9617,-76.9511,86.83,86.83,67.0,89.0,10.47
716,Jinxiang,CN,1.627416e+09,27.4327,120.6062,80.67,80.67,94.0,100.0,8.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure the gmaps
gmaps.configure(api_key = g_key)

# Store the Latitude and Longitude in Locations
locations = weather_cities_data[["Latitude", "Longitude"]]
weight = weather_cities_data["Humidity"]

# create heat layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = weight,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values
new_city_df = weather_cities_data.loc[(weather_cities_data["Wind Speed"] <= 10) & (weather_cities_data["Clouds"] == 0) & \
                                      (weather_cities_data["Max Temperature"] >= 70) & (weather_cities_data["Max Temperature"] <= 80)].dropna()
new_city_df.dropna(inplace = True) 
new_city_df

,City,Country,Date,Latitude,Longitude,Temperature,Max Temperature,Humidity,Clouds,Wind Speed
1,East London,ZA,1.627416e+09,-33.0153,27.9116,73.15,73.15,30.0,0.0,8.08
51,Umeå,SE,1.627416e+09,63.8284,20.2597,71.53,71.53,49.0,0.0,3.44
65,Province of Vibo Valentia,IT,1.627416e+09,38.6996,16.1274,74.26,79.65,61.0,0.0,1.48
124,Tiznit Province,MA,1.627416e+09,29.5833,-9.5000,71.46,71.46,70.0,0.0,4.25
159,Ankara,TR,1.627416e+09,39.9199,32.8543,73.42,73.42,44.0,0.0,8.86
178,Aichach,DE,1.627416e+09,48.4576,11.1341,68.92,72.45,79.0,0.0,3.44
205,Kushikino,JP,1.627416e+09,31.7167,130.2667,74.16,74.16,85.0,0.0,3.04
230,Zaozyornoye,UA,1.627416e+09,45.1582,33.2787,76.95,76.95,58.0,0.0,5.70
341,Zharkent,KZ,1.627416e+09,44.1628,80.0000,75.87,75.87,39.0,0.0,1.23
348,Pimentel,PE,1.627416e+09,-6.8367,-79.9342,71.01,73.69,79.0,0.0,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# create hotel_df 
hotel_df = new_city_df.loc[:, ["City", "Country", "Latitude", "Longitude"]]
# add hotel name to hotel_df
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,East London,ZA,-33.0153,27.9116,
51,Umeå,SE,63.8284,20.2597,
65,Province of Vibo Valentia,IT,38.6996,16.1274,
124,Tiznit Province,MA,29.5833,-9.5000,
159,Ankara,TR,39.9199,32.8543,
178,Aichach,DE,48.4576,11.1341,
205,Kushikino,JP,31.7167,130.2667,
230,Zaozyornoye,UA,45.1582,33.2787,
341,Zharkent,KZ,44.1628,80.0000,
348,Pimentel,PE,-6.8367,-79.9342,


In [10]:
# print the name of the hotel for each city
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# set up a parameteres dictonary
params = {"type": "hotel",
          "Keyword": "hotel",
          "radius": 5000,
          "Key": g_key   
}
# loop through the hotel_df and run a lat/long search for each city
for index, row in hotel_df.iterrows():   
    #get the city name, lat, and longtitude from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    city_name = row["City"]
#     add ketword to params dict
    params["Location"] = f"{latitude},{longitude}"
# create url and make API calls
    print(f"Retriving Results for Index {index}: {city_name}.")
# get the response
    response = requests.get(base_url, params=params).json()
#     get the results from the response
    results = response["results"]
    
    try:
        print(f"Closet hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
            print("Missing field/result ..... skipping.")
            
    print("-----------")
print("--------End of Search ----------")
        


Retriving Results for Index 1: East London.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 51: Umeå.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 65: Province of Vibo Valentia.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 124: Tiznit Province.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 159: Ankara.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 178: Aichach.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 205: Kushikino.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 230: Zaozyornoye.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 341: Zharkent.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 348: Pimentel.
Missing field/result ..... skipping.
-----------
Retriving Results for Index 397: Salamiyah.
Missing field/re

In [11]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,East London,ZA,-33.0153,27.9116,
51,Umeå,SE,63.8284,20.2597,
65,Province of Vibo Valentia,IT,38.6996,16.1274,
124,Tiznit Province,MA,29.5833,-9.5000,
159,Ankara,TR,39.9199,32.8543,
178,Aichach,DE,48.4576,11.1341,
205,Kushikino,JP,31.7167,130.2667,
230,Zaozyornoye,UA,45.1582,33.2787,
341,Zharkent,KZ,44.1628,80.0000,
348,Pimentel,PE,-6.8367,-79.9342,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))